# Dijkstra


In [1]:
import sys
sys.path.append("..")

In [1]:
from hoteling.graph_generators import generate_grid_graph

g = generate_grid_graph(3, 4)

In [2]:
from hoteling.branch_bound import BBTree, BaseRevenueFunction, Node


base_cost = 3
revenue_function = BaseRevenueFunction(base_cost)

M = 12
bb_tree = BBTree(g, M, revenue_function)
bb_tree.run(max_iterations=100)
bb_tree.run_stat

runStatistics(open_nodes=147, reused_nodes=159, rejected_nodes=101)

In [3]:
positions = Node.get_positions(
    g,
    (5, 6, 7),
    M,
    "weight",
    revenue_function,extended_return=True)
positions

{'num_sellers': {5: 6, 6: 3, 7: 3},
 'lambd': np.float64(1.8333333333339166),
 'nearest_set': <VertexPropertyMap object with value type 'python::object', for Graph 0x122ac25a0, at 0x1374fdd90>,
 'distances': <VertexPropertyMap object with value type 'double', for Graph 0x122ac25a0, at 0x1374ff3b0>,
 'revenues': defaultdict(float, {5: 11.0, 6: 7.0, 7: 7.0})}

In [5]:
from hoteling.plot_utils.widget import panel_gt_interactive
from hoteling.graph_generators import generate_grid_graph
from hoteling.branch_bound import BBTree, BaseRevenueFunction, Node

g = generate_grid_graph(3, 4)

base_cost = 3
revenue_function = BaseRevenueFunction(base_cost)

panel_gt_interactive(g, revenue_function)